### Расчет Fcost

In [22]:
import numpy as np
import cv2
import os
from skimage.feature import graycomatrix, graycoprops, local_binary_pattern
from scipy.spatial.distance import euclidean
from glob import glob

In [23]:
# --- Параметры ---
LBP_RADIUS = 1
LBP_N_POINTS = 8 * LBP_RADIUS
LBP_METHOD = 'uniform'
W_C = 100

In [24]:
# --- Функции извлечения признаков ---
def compute_glcm_features(image):
    image_u8 = (image * 255).astype(np.uint8)
    glcm = graycomatrix(image_u8, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    props = ['contrast', 'dissimilarity', 'homogeneity']
    return np.array([graycoprops(glcm, p)[0, 0] for p in props])


def compute_lbp_histogram(image):
    lbp = local_binary_pattern(image, LBP_N_POINTS, LBP_RADIUS, method=LBP_METHOD)
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, LBP_N_POINTS + 3),
                           range=(0, LBP_N_POINTS + 2), density=True)
    return hist[:3]  # первые 3 признака


def compute_gradient_features(image):
    grad_x = cv2.Sobel(image, cv2.CV_64F, 1, 0)
    grad_y = cv2.Sobel(image, cv2.CV_64F, 0, 1)
    magnitude = np.sqrt(grad_x**2 + grad_y**2)
    return np.array([np.mean(magnitude), np.std(magnitude), np.max(magnitude)])


# --- Загружаем фрагменты ---
def load_fragments(folder):
    paths = sorted(glob(os.path.join(folder, '*.png')))
    fragments = [cv2.imread(p, cv2.IMREAD_GRAYSCALE) / 255.0 for p in paths]
    return fragments


# --- Достоверность: бинарная маска оригинальной области (не экстаполированная) ---
def compute_confidence_map(fragment):
    return (fragment > 0.05).astype(float)


# --- Преобразование: здесь тождественное ---
def transform(feature, T):
    return feature

In [25]:
def compute_f_cost(fragments, w_c=1000):
    n_p = len(fragments)
    n_k = 3
    Th_k = np.array([0.5, 0.5, 0.5])
    w_k = 1 / Th_k

    features = np.zeros((n_p, n_k, 3))
    confidences = np.zeros((n_p, n_k))
    L_map = []

    for i, img in enumerate(fragments):
        features[i][0] = compute_glcm_features(img)
        features[i][1] = compute_lbp_histogram(img)
        features[i][2] = compute_gradient_features(img)
        c_map = compute_confidence_map(img)
        confidences[i] = [np.mean(c_map)] * n_k
        L_map.append((c_map > 0.5).astype(int))

    # --- m1: сходство текстур ---
    total_similarity = 0.0
    total_confidence = 0.0
    for i in range(n_p):
        for j in range(i+1, n_p):
            for k in range(n_k):
                D = euclidean(features[i][k], features[j][k])
                S = w_k[k] * (D - 1)
                C_i = confidences[i][k]
                C_j = confidences[j][k]
                total_similarity += S * C_i * C_j
                total_confidence += C_i

    m1 = total_similarity / total_confidence if total_confidence != 0 else 0

    # --- m2: геометрические ограничения (пересечения) ---
    m2 = 0
    for i in range(n_p):
        for j in range(i+1, n_p):
            overlap = np.logical_and(L_map[i], L_map[j])
            m2 += np.sum(overlap)

    F_cost = m1 + w_c * m2

    print(f'm1 = {m1}, m2 * w_c = {w_c*m2}, m2 = {m2}')

    return F_cost


Пример использования

In [26]:
fragments = load_fragments('pics/split_results/mask2')  # <- путь к твоим фрагментам
cost = compute_f_cost(fragments, w_c=W_C)
print(f"F_cost = {cost:.4f}")

m1 = 1.3683052580812403, m2 * w_c = 0, m2 = 0
F_cost = 1.3683


/opt/miniconda3/lib/python3.12/site-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(
